In [ ]:
import geopandas as gpd
import pandas as pd

In [5]:
# 공간 데이터 csv로 변환하여 저장
# gdf = gpd.read_file('/Users/Jenny/Desktop/공모전/서울시 주요시설과 집객시설 공간데이터/2. 파일데이터/2020/TL_TBVIATR_FCLTY_INFO_2020.shp')
# csv_file = "서울시 주요시설과 집객시설 공간데이터(2020).csv"
# gdf.to_csv(csv_file, index=False)


## 생략가능 아래 코드만 실행하면 됩니다

In [15]:
# 엑셀을 이용해 가볍게 1차 전처리 한 파일을 불러오기

seoul = pd.read_csv('서울시 주요시설과 집객시설 공간데이터(2020).csv')
seoul.head()

,MLSFCDC,VIAT_NM,X_VALUE,Y_VALUE
0,교육시설,"강덕초교,고덕그라시움버스정류장쉘터",214388.672913,551205.841919
1,서비스산업,고덕그라시움상가,214555.910095,550962.356812
2,교육시설,서울강덕초등학교,214224.118713,550979.324829
3,교육시설,은빛어린이집,214171.512024,550841.469116
4,서비스산업,고덕그라시움상가,214481.776611,550819.438110


In [16]:
# 교육시설 제외하기
seoul = seoul[seoul['MLSFCDC'] != '교육시설']
seoul = seoul.reset_index(drop=True)
seoul.head()

,MLSFCDC,VIAT_NM,X_VALUE,Y_VALUE
0,서비스산업,고덕그라시움상가,214555.910095,550962.356812
1,서비스산업,고덕그라시움상가,214481.776611,550819.438110
2,서비스산업,그라시움상가,214465.779480,551177.935486
3,문화/예술시설,송파BOOK책박물관,209236.807129,544386.432007
4,서비스산업,"가락시장,헬리오시티버스정류장쉘터",209386.103088,543972.747620


In [17]:
# X,Y좌표 경도 위도로 변환하기
gdf = gpd.GeoDataFrame(seoul, geometry=gpd.points_from_xy(seoul.X_VALUE, seoul.Y_VALUE))

gdf.crs = 'epsg:5186'
gdf = gdf.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y
gdf = gdf[['MLSFCDC', 'VIAT_NM', 'x', 'y']]

print(gdf)

       MLSFCDC            VIAT_NM           x          y
0        서비스산업           고덕그라시움상가  127.164740  37.558074
1        서비스산업           고덕그라시움상가  127.163898  37.556787
2        서비스산업             그라시움상가  127.163724  37.560018
3      문화/예술시설         송파BOOK책박물관  127.104457  37.498893
4        서비스산업  가락시장,헬리오시티버스정류장쉘터  127.106140  37.495165
...        ...                ...         ...        ...
12985    서비스산업                NaN  126.894531  37.472180
12986     보건시설                NaN  126.909474  37.450467
12987    서비스산업                NaN  126.902265  37.471291
12988     보건시설                NaN  126.872023  37.524689
12989     보건시설                NaN  127.033093  37.527229

[12990 rows x 4 columns]


In [12]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [18]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in gdf.iterrows():
    get_location(row)
    
    # 200개씩 처리될 때 메시지 출력
    if i % 200 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
gdf['gu'] = [location.get('gu') for location in locations]
gdf['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2800 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보 불러오는 중...
3200 번째 경위도 좌표계에 대한 정보 불러오는 중...
3400 번째 경위도 좌표계에 대한 정보 불러오는 중...
3600 번째 경위도 좌표계에 대한 정보 불러오는 중...
3800 번째 경위도 좌표계에 대한 정보 불러오는 중...
4000 번째 경위도 좌표계에 대한 정보 불러오는 중...
4200 번째 경위도 좌표계에 대한 정보 불러오는 중...
4400 번째 경위도 좌표계에 대한 정보 불러오는 중...
4600 번째 경위도 좌표계에 대한 정보 불러오는 중...
4800 번째 경위도 좌표계에 대한 정보 불러오는 중...
5000 번째 경위도 좌표계에 대한 정보 불러오는 중...
5200 번째 경위도 좌표계에 대한 정보 불러오는 중...
5400 번째 경위도 좌표계에 대한 정보 불러오는 중...
5600 번째 경위도 좌표계에 대한 정보 불러오는 중...
5800 번째 경위도 좌표계에 대한 정보 불러오는 중...
6000 번째 경위도 좌표계에 

In [21]:
gdf.to_csv("서울시 주요시설과 집객시설 공간데이터(2020).csv")